# Pos-Tagging & Feature Extraction
Following normalisation, we can now proceed to the process of pos-tagging and feature extraction. Let's start with pos-tagging.

## POS-tagging
Part-of-speech tagging is one of the most important text analysis tasks used to classify words into their part-of-speech and label them according the tagset which is a collection of tags used for the pos tagging. Part-of-speech tagging also known as word classes or lexical categories.

The `nltk` library provides its own pre-trained `POS-tagger`. Let's see how it is used. 

In [5]:
import pandas as pd
df0=pd.read_csv("../data/interim/normalised_keyed_reviews_100k_sample.csv", sep="\t", low_memory=False)
df0.head()

,uniqueKey,reviewText
0,A10000012B7CGYKOMPQ4L##000100039X,"['spiritually', 'mentally', 'inspiring', 'book..."
1,A2S166WSCFIFP5##000100039X,"['one', 'must', 'books', 'masterpiece', 'spiri..."
2,A1BM81XB4QHOA3##000100039X,"['book', 'provides', 'reflection', 'apply', 'l..."
3,A1MOSTXNIO5MPJ##000100039X,"['first', 'read', 'prophet', 'college', 'back'..."
4,A2XQ5LZHTD4AFT##000100039X,"['timeless', 'classic', 'demanding', 'assuming..."


In [8]:
# For monitoring duration of pandas processes
from tqdm import tqdm, tqdm_pandas

# To avoid RuntimeError: Set changed size during iteration
tqdm.monitor_interval = 0

# Register `pandas.progress_apply` and `pandas.Series.map_apply` with `tqdm`
# (can use `tqdm_gui`, `tqdm_notebook`, optional kwargs, etc.)
tqdm.pandas(desc="Progress:")

# Now you can use `progress_apply` instead of `apply`
# and `progress_map` instead of `map`
# can also groupby:
# df.groupby(0).progress_apply(lambda x: x**2)

In [ ]:
import nltk

tagged_df = pd.DataFrame(df0['reviewText'].progress_apply(lambda review: nltk.pos_tag(review)))
tagged_df.head()

Progress::   0%|          | 8657/8898041 [04:09<111:27:29, 22.15it/s]

`nltk` provides documentation for each tag, which can be queried using the tag, e.g., `nltk.help.upenn_tagset(‘RB’)`, or a regular expression. `nltk` also provides batch pos-tagging method for document pos-tagging:
